In [ ]:
# %pip install langchain_openai
# %pip install langchain_community
# %pip install google-search-results

In [1]:
from config import CustomAgent,WebScrap,extract_text_from_url
from prompt import VerticalPrompt
import json
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

#### Web Scrap Content

In [3]:
content = extract_text_from_url("https://www.datamatter.tech/")
content

"top of page TM Home Solution Resources Company Contact Us More Use tab to navigate through the menu items. The New Standard in Data Management We offer end-to-end data migration solutions that make data analysis and management a breeze for your business. Get a 360-degree view and 100% control of your data migration project with our solution. Learn More Beyond ETL Our product distinguishes itself from competitors through its commitment to simplicity, end-to-end solution delivery, and a strong focus on the end-user experience. Here's how we stand out \u200b Additionally, we recognize the importance of efficiency and accuracy in data management. Simplicity We prioritize an intuitive application design, ensuring users of varying technical backgrounds can effortlessly navigate and fully leverage our platform. This approach significantly reduces the learning curve and maximizes user engagement and productivity. End-to-End Delivery Our platform offers a holistic data migration solution, cove

In [4]:
WebSccrap = WebScrap()
web_content=WebSccrap.invoke_web_scraping(webcontent=content,model_name="google",model_variant="gemini-1.5-flash")
web_content

Web Scrap Agent Initialized✅


'Datamatter™ Technologies operates in the **data management and migration solutions** domain.  They offer an end-to-end platform designed for simplicity and ease of use, focusing on automation and a strong user experience.  Their solutions aim to streamline data analysis and management for businesses, improving efficiency and accuracy.  The company emphasizes features such as automated tracking, progress reporting, and automation of repetitive processes.'

In [5]:
prompt = VerticalPrompt()

In [6]:
prompt_txt=prompt. get_prompt(client_name="DataMatter",client_description="A leading provider of migration services",website_content=web_content)
prompt_txt

'\n    Identify the primary business domain for the following client:\n    Client Name: DataMatter\n        Client Description: A leading provider of migration services\n        Website Content: Datamatter™ Technologies operates in the **data management and migration solutions** domain.  They offer an end-to-end platform designed for simplicity and ease of use, focusing on automation and a strong user experience.  Their solutions aim to streamline data analysis and management for businesses, improving efficiency and accuracy.  The company emphasizes features such as automated tracking, progress reporting, and automation of repetitive processes.\n\n    Based on this information, classify the client into one of the following business domains:\n        - List your predefined business domains here, e.g., Healthcare, Finance, Retail, Manufacturing, Technology, Education, etc.\n        \n        Provide the following in your response in JSON format:\n    \n            "domain_name": "Identif

#### *initialize Agent*

In [7]:
VerticalAgent = CustomAgent()

Vertical Agent Initialized✅


In [8]:
# get the list of models
models = VerticalAgent.model_list()
print(models)

['google', 'openai']


In [9]:
# get the list of output parsers
output_parsers = VerticalAgent.output_parser_list()
print(output_parsers)

['str', 'json', 'pydantic']


In [10]:
# open json file get prompt
with open('Prompt.json') as file:
    data = json.load(file)

In [11]:
# user_input = data[0].get('user_input')
system_input = data[0].get('system_msg')
# print(user_input)  
print(system_input)

You are an AI assistant specializing in providing detailed information about various business verticals. Based on the user's input, you will generate a structured response, summarizing the industry and domain the user mentions. '


In [62]:
response = VerticalAgent.invoke_agent(
    user_input= prompt_txt,
    system_input= system_input,
    model_name="google",
    output_parser="json",
    verbose=True,
    model_variant="gemini-1.5-flash"
)



> Entering new AgentExecutor chain...
Thought:The question asks to identify the business domain of DataMatter based on the provided description. The description clearly indicates that DataMatter operates in the data management and migration solutions domain.  I don't need to perform a web search. I can directly generate the JSON response.

Thought: I now know the final answer

Final Answer:
```json
{
  "domain_name": "Technology",
  "domain_description": "DataMatter operates in the technology sector, specifically providing data management and migration solutions.  Their services focus on automating data processes, improving efficiency and accuracy for businesses.",
  "keywords": ["data management", "data migration", "data analysis", "automation", "software", "technology", "IT services", "cloud", "migration services", "data solutions"],
  "confidence_score": 0.98
}
```

> Finished chain.


In [63]:
print("response :",response)

response : {'domain_name': 'Technology', 'domain_description': 'DataMatter operates in the technology sector, specifically providing data management and migration solutions.  Their services focus on automating data processes, improving efficiency and accuracy for businesses.', 'keywords': ['data management', 'data migration', 'data analysis', 'automation', 'software', 'technology', 'IT services', 'cloud', 'migration services', 'data solutions'], 'confidence_score': 0.98}


In [64]:
print("type of response :",type(response))

type of response : <class 'dict'>


In [65]:
json_data = response
print(json_data)

{'domain_name': 'Technology', 'domain_description': 'DataMatter operates in the technology sector, specifically providing data management and migration solutions.  Their services focus on automating data processes, improving efficiency and accuracy for businesses.', 'keywords': ['data management', 'data migration', 'data analysis', 'automation', 'software', 'technology', 'IT services', 'cloud', 'migration services', 'data solutions'], 'confidence_score': 0.98}


In [66]:
file_path = 'VerticalAgent_Output.json'


In [67]:
# Check if the file exists
if os.path.exists(file_path):
    # Read the existing data from the file
    with open(file_path, 'r') as file:
        existing_data = json.load(file)
        # If the existing data is not a list, make it one
        if not isinstance(existing_data, list):
            existing_data = [existing_data]
else:
    # If the file doesn't exist, start with an empty list
    existing_data = []

# Append the new response to the existing data (which should now be a list)
existing_data.append(response)


# Write the updated data back to the file
with open(file_path, 'w') as file:
    json.dump(existing_data, file, indent=4)

### database


In [68]:
# %pip install sentence-transformers psycopg2 numpy


In [69]:
import psycopg2
import numpy as np
from sentence_transformers import SentenceTransformer

In [70]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")  # 384-d embeddings

# PostgreSQL connection
conn = psycopg2.connect(
    database="vect",
    user="postgres",
    password="newpassword",
    host="127.0.0.1",
    port=5432
)
cursor = conn.cursor()

In [71]:
# Ensure pgvector extension is enabled
cursor.execute("CREATE EXTENSION IF NOT EXISTS vector")

In [72]:
# Create table if not exists
cursor.execute("""
    CREATE TABLE IF NOT EXISTS domain_embeddings (
        id SERIAL PRIMARY KEY,
        domain_name TEXT,
        domain_description TEXT,
        keywords TEXT[],
        confidence_score FLOAT,
        embedding vector(384) -- Change to match the model's output size
    )
""")
conn.commit()

In [73]:
# # Sample JSON from LLM
# json_data = {
#     "domain_name": "Healthcare",
#     "domain_description": "The client is a leading provider of healthcare services and medical devices worldwide, indicating a primary focus on the healthcare industry.",
#     "keywords": [
#         "healthcare",
#         "medical devices",
#         "services",
#         "hospital",
#         "clinic",
#         "pharmaceutical",
#         "medical supplies",
#         "treatment"
#     ],
#     "confidence_score": 0.95
# }

In [74]:
# Convert numerical value to string
text_to_embed = (
    json_data["domain_name"] + " " +
    json_data["domain_description"] + " " + 
    " ".join(json_data["keywords"]) + " " + "Confidance Score "+ 
    str(json_data["confidence_score"])  # Convert float to string before concatenation
)


In [75]:
text_to_embed

'Technology DataMatter operates in the technology sector, specifically providing data management and migration solutions.  Their services focus on automating data processes, improving efficiency and accuracy for businesses. data management data migration data analysis automation software technology IT services cloud migration services data solutions Confidance Score 0.98'

In [76]:
# Generate embedding using Hugging Face
embedding = model.encode(text_to_embed).tolist()

In [77]:
len(embedding)

384

In [79]:
cursor.execute("""
    INSERT INTO domain_embeddings (domain_name, domain_description, keywords, confidence_score, embedding)
    VALUES (%s, %s, %s, %s, %s)
""", (
    json_data["domain_name"],
    json_data["domain_description"],
    json_data["keywords"],
    json_data["confidence_score"],
    embedding
))

conn.commit()
cursor.close()
conn.close()
print("✅ Data inserted successfully using Hugging Face embeddings!")

InterfaceError: cursor already closed